In [2]:
import time
start_time = time.time()
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd
np.random.seed(32)
os.environ["OMP_NUM_THREADS"] = "4"
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [37]:
num_models = 5
kf = KFold(n_splits=num_models, random_state=2222)

In [4]:
def get_oof(clf, X_train, y_train, X_test):
    oof_train = np.zeros((X_train.shape[0], 6))
    oof_test = np.zeros((num_models, X_test.shape[0],6))
    
    for i, (train_index, val_index) in enumerate(kf.split(X_train)):
        print("Fold: {0}".format(i))
        kf_X_train = X_train[train_index]
        kf_y_train = y_train[train_index]
        kf_X_val = X_train[val_index]
        
        print(kf_X_train.shape)
        
        clf.fit(kf_X_train, kf_y_train)
        
        oof_train[val_index,:] = clf.predict(kf_X_val)
        oof_test[i,:,:] = clf.predict(X_test)
        
    oof_test = np.mean(oof_test, axis=0)
    assert oof_test.shape == (X_test.shape[0],6)
    assert oof_train.shape == (X_train.shape[0], 6)
    
    return oof_train, oof_test
        

In [38]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [39]:
train = pd.read_csv("./dataset/train.csv", header = 0)
test = pd.read_csv("./dataset/test.csv", header = 0)
embedding_path = "./dataset/crawl-300d-2M.vec"

In [40]:
embed_size = 300
max_features = 100000
max_len = 150

In [41]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = train[list_classes].values

train["comment_text"].fillna("no comment")
test["comment_text"].fillna("no comment")

X_train = train["comment_text"].str.lower()
X_test = test["comment_text"].str.lower() 

In [10]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

In [12]:
word_index = tk.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [42]:
tk = Tokenizer(num_words = max_features, lower = True)
tk.fit_on_texts(list(X_train)+list(X_test))   

X_train = tk.texts_to_sequences(X_train)
X_test = tk.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [43]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
 
        
def build_model(X_test,lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    
    oof_train = np.zeros((X_train.shape[0], 6))
    oof_test = np.zeros((num_models, X_test.shape[0],6))
    
    for i, (train_index, val_index) in enumerate(kf.split(X_train)):
        print("Fold: {0}".format(i))
        
        
        kf_X_train = X_train[train_index]
        kf_y_train = y_train[train_index]
        kf_X_val= X_train[val_index]
        kf_y_val = y_train[val_index]
        
        print(kf_X_train.shape)
      
        ra_val = RocAucEvaluation(validation_data=(kf_X_val, kf_y_val), interval = 1)
    
        #model
        inp = Input(shape = (max_len,))
        x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
        x = SpatialDropout1D(dr)(x)

        x = Bidirectional(GRU(units, return_sequences = True))(x)
        x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
        avg_pool = GlobalAveragePooling1D()(x)
        max_pool = GlobalMaxPooling1D()(x)
        x = concatenate([avg_pool, max_pool])

        x = Dense(6, activation = "sigmoid")(x)
        model = Model(inputs = inp, outputs = x)
        model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
        print(i)
    
        model.fit(kf_X_train,kf_y_train,batch_size = 128, epochs = 4, validation_data=(kf_X_val, kf_y_val), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop])
        model = load_model(file_path)
        
        oof_train[val_index,:] = model.predict(kf_X_val, verbose = 1)
        oof_test[i,:,:] = model.predict(X_test, verbose = 1)
        
    oof_test = np.mean(oof_test, axis=0)
    assert oof_test.shape == (X_test.shape[0],6)
    assert oof_train.shape == (X_train.shape[0], 6)
    
    return oof_train, oof_test

In [ ]:
oof_train, oof_test = build_model(X_test,lr = 1e-3, lr_d = 0, units = 128, dr = 0.2)

In [45]:
oof_train

array([[6.78710919e-03, 2.08987869e-04, 2.56251567e-03, 2.74572885e-05,
        1.88121700e-03, 1.77073045e-04],
       [5.23041061e-04, 5.20681751e-05, 2.27438781e-04, 2.45440424e-05,
        4.99723887e-04, 6.69919318e-05],
       [5.28604025e-03, 1.72453612e-04, 5.05819509e-04, 3.15974394e-05,
        5.84439840e-04, 1.79916795e-04],
       ...,
       [1.55865084e-02, 9.61945392e-04, 5.88781899e-03, 7.28134983e-05,
        1.51336344e-03, 1.35139460e-04],
       [1.67419354e-03, 7.52802152e-05, 4.40404052e-04, 1.42882374e-04,
        6.32530719e-04, 3.03306279e-05],
       [1.91281289e-02, 4.78021742e-04, 2.75466172e-03, 1.07453507e-03,
        2.81989621e-03, 1.22814105e-04]])

In [46]:
submission = pd.read_csv("./dataset/sample_submission.csv")
submission[list_classes] = (oof_test)
submission.to_csv("submission_stack_test.csv", index = False)
print("[{}] Completed!".format(time.time() - start_time))

[40462.89393281937] Completed!


In [47]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.991794,0.488682,0.967501,0.258997,0.942821,0.387452
1,0000247867823ef7,0.001009,0.000159,0.000341,0.000026,0.000547,0.000031
2,00013b17ad220c46,0.001770,0.000536,0.000567,0.000099,0.000682,0.000121
3,00017563c3f7919a,0.000315,0.000030,0.000161,0.000101,0.000192,0.000012
4,00017695ad8997eb,0.011360,0.000913,0.002530,0.000521,0.001644,0.000120


In [48]:
train = pd.read_csv("./dataset/train.csv", header = 0)
train_id = train["id"]
submission = pd.DataFrame(columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
submission["id"] = train_id
submission[list_classes] = (oof_train)
submission.to_csv("submission_stack_train.csv", index = False)